In [8]:
import mlflow
import mlflow.sklearn
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np
import pickle
import pathlib
import pandas as pd

In [9]:
df_train = pd.read_parquet('data/green_tripdata_2024-01.parquet')
df_val = pd.read_parquet('data/green_tripdata_2024-02.parquet')

In [10]:
df_train['duration'] = (df_train['lpep_dropoff_datetime'] - df_train['lpep_pickup_datetime']).dt.total_seconds() / 60
df_val['duration'] = (df_val['lpep_dropoff_datetime'] - df_val['lpep_pickup_datetime']).dt.total_seconds() / 60

X_train = df_train[['trip_distance']]
y_train = df_train['duration']
X_val = df_val[['trip_distance']]
y_val = df_val['duration']

In [11]:
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

best_rmse = float("inf")
best_model = None

In [12]:
def objective_gb(params):
    global best_rmse, best_model
    with mlflow.start_run(nested=True):
        model = GradientBoostingRegressor(**params)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)
        score = rmse(y_val, y_pred)
        
        mlflow.log_params(params)
        mlflow.log_metric('rmse', score)
        mlflow.sklearn.log_model(model, artifact_path="model")
        
        if score < best_rmse:
            best_rmse = score
            best_model = model  # Guardar el mejor modelo
        
        return {'loss': score, 'status': STATUS_OK}

In [13]:
search_space_gb = {
    'n_estimators': scope.int(hp.quniform('n_estimators', 50, 500, 25)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'max_depth': scope.int(hp.quniform('max_depth', 2, 10, 1)),
    'min_samples_split': scope.int(hp.quniform('min_samples_split', 2, 10, 1)),
    'min_samples_leaf': scope.int(hp.quniform('min_samples_leaf', 1, 10, 1)),
    'random_state': 42
}

In [14]:
with mlflow.start_run(run_name="GradientBoostingExperiment"):
    trials_gb = Trials()
    best_params_gb = fmin(
        fn=objective_gb,
        space=search_space_gb,
        algo=tpe.suggest,
        max_evals=20,
        trials=trials_gb
    )
    mlflow.log_params(best_params_gb)

  0%|          | 0/20 [00:00<?, ?trial/s, best loss=?]

2024/09/20 13:13:55 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



  5%|▌         | 1/20 [00:20<06:27, 20.37s/trial, best loss: 79.6131056874737]

2024/09/20 13:14:03 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



 10%|█         | 2/20 [00:28<03:53, 12.95s/trial, best loss: 78.0291897411658]

...

 90%|█████████ | 18/20 [04:40<00:27, 13.66s/trial, best loss: 77.24229901339169]

2024/09/20 13:18:37 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



 95%|█████████▌| 19/20 [05:02<00:16, 16.23s/trial, best loss: 77.24229901339169]

2024/09/20 13:18:53 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



100%|██████████| 20/20 [05:17<00:00, 15.90s/trial, best loss: 77.24229901339169]


In [15]:
def objective_rf(params):
    global best_rmse, best_model
    with mlflow.start_run(nested=True):
        model = RandomForestRegressor(**params)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)
        score = rmse(y_val, y_pred)
        
        mlflow.log_params(params)
        mlflow.log_metric('rmse', score)
        mlflow.sklearn.log_model(model, artifact_path="model")
        
        if score < best_rmse:
            best_rmse = score
            best_model = model  # Guardar el mejor modelo
        
        return {'loss': score, 'status': STATUS_OK}

In [16]:
search_space_rf = {
    'n_estimators': scope.int(hp.quniform('n_estimators', 50, 500, 25)),
    'max_depth': scope.int(hp.quniform('max_depth', 2, 20, 1)),
    'min_samples_split': scope.int(hp.quniform('min_samples_split', 2, 10, 1)),
    'min_samples_leaf': scope.int(hp.quniform('min_samples_leaf', 1, 10, 1)),
    'random_state': 42
}

In [17]:
with mlflow.start_run(run_name="RandomForestExperiment"):
    trials_rf = Trials()
    best_params_rf = fmin(
        fn=objective_rf,
        space=search_space_rf,
        algo=tpe.suggest,
        max_evals=20,
        trials=trials_rf
    )
    mlflow.log_params(best_params_rf)

  0%|          | 0/20 [00:00<?, ?trial/s, best loss=?]

2024/09/20 13:20:13 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



  5%|▌         | 1/20 [00:30<09:36, 30.35s/trial, best loss: 77.81407600837389]

2024/09/20 13:20:41 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



 10%|█         | 2/20 [00:58<08:44, 29.14s/trial, best loss: 77.81407600837389]

...

 90%|█████████ | 18/20 [04:41<00:19,  9.50s/trial, best loss: 77.14146726597599]

2024/09/20 13:24:30 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



 95%|█████████▌| 19/20 [04:47<00:08,  8.48s/trial, best loss: 77.14146726597599]

2024/09/20 13:24:39 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



100%|██████████| 20/20 [04:57<00:00, 14.86s/trial, best loss: 77.14146726597599]


In [18]:
with mlflow.start_run(run_name="Register Best Model"):
    mlflow.sklearn.log_model(best_model, artifact_path="model", registered_model_name="nyc-taxi-model")
    print(f"El modelo con mejor RMSE ({best_rmse}) ha sido registrado.")

2024/09/20 13:25:30 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'nyc-taxi-model'.
Created version '1' of model 'nyc-taxi-model'.


El modelo con mejor RMSE (77.14146726597599) ha sido registrado.
